<a href="https://colab.research.google.com/github/avikumart/LLM-GenAI-Transformers-Notebooks/blob/main/Semantic_search_using_ChromaDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install chromadb -q
!pip install sentence-transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 56.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 966.7/966.7 kB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 93.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 93.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 57.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 73.8 MB/

**ChromaDB docs:** [LInk](https://docs.trychroma.com/)

In [2]:
# import chromadb and create client
import chromadb

client = chromadb.Client()
collection = client.create_collection("my-collection")

In [3]:
# add the documents in the db
collection.add(
    documents=["This is a document about cat", "This is a document about car", "This is a document about bike"],
    metadatas=[{"category": "animal"}, {"category": "vehicle"}, {"category": "vehicle"}],
    ids=["id1", "id2","id3"]
)

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:02<00:00, 30.8MiB/s]


In [5]:
# ask the querying to retrieve the data from DB
results = collection.query(
    query_texts=["animal"],
    n_results=1
)
print(results)

{'ids': [['id1']], 'embeddings': None, 'documents': [['This is a document about cat']], 'metadatas': [[{'category': 'animal'}]], 'distances': [[1.157336950302124]]}


### Semantic search using ChromaDB

In [8]:
# import files from the pets folder to store in VectorDB
import os

def read_files_from_folder(folder_path):
    file_data = []

    for file_name in os.listdir(folder_path):
        if file_name.endswith(".txt"):
            with open(os.path.join(folder_path, file_name), 'r') as file:
                content = file.read()
                file_data.append({"file_name": file_name, "content": content})

    return file_data

folder_path = "/content/pets"
file_data = read_files_from_folder(folder_path)

In [11]:
# get the data from file_data and create chromadb collection
documents = []
metadatas = []
ids = []

for index, data in enumerate(file_data):
    documents.append(data['content'])
    metadatas.append({'source': data['file_name']})
    ids.append(str(index + 1))

pet_collection = client.create_collection("pet_collection")

pet_collection.add(
    documents=documents,
    metadatas=metadatas,
    ids=ids
)

In [14]:
metadatas

[{'source': 'Nutrition Needs of Pet Animals.txt'},
 {'source': 'Health Care for Pets.txt'},
 {'source': 'The Emotional Bond Between Humans and Pets.txt'},
 {'source': 'Different Types of Pet Animals.txt'},
 {'source': 'Training and Behaviour of Pets.txt'}]

In [16]:
# query the database to get the answer from vectorized data
results = pet_collection.query(
    query_texts=["What is the Nutrition needs of the pet animals?"],
    n_results=1
)

results

{'ids': [['1']],
 'embeddings': None,
 'documents': [['Proper nutrition is vital for the health and wellbeing of pets. Dogs and cats require a balanced diet that includes proteins, carbohydrates, and fats. Some may even have specific dietary needs based on their breed or age. Birds typically thrive on a diet of seeds, fruits, and vegetables, while reptiles have diverse diets ranging from live insects to fresh produce. Fish diets depend greatly on the species, with some needing live food and others subsisting on flakes or pellets.']],
 'metadatas': [[{'source': 'Nutrition Needs of Pet Animals.txt'}]],
 'distances': [[0.5794490575790405]]}

In [21]:
# filtering the data based on some key words
pet_collection.query(
    query_texts=["What are the emotional benefits of owning a pet?"],
    n_results=1,
     where={"source": "Training and Behaviour of Pets.txt"}
)

{'ids': [['5']],
 'embeddings': None,
 'documents': [['Training is essential for a harmonious life with pets, particularly for dogs. It helps pets understand their boundaries and makes cohabitation easier for both pets and owners. Training should be based on positive reinforcement. Understanding pet behavior is also important, as changes in behavior can often be a sign of underlying health issues.']],
 'metadatas': [[{'source': 'Training and Behaviour of Pets.txt'}]],
 'distances': [[0.8881877064704895]]}

### Semantic search application using huggingface model

In [22]:
# import the sentence transformers
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('paraphrase-MiniLM-L3-v2')

documents = []
embeddings = []
metadatas = []
ids = []

for index, data in enumerate(file_data):
    documents.append(data['content'])
    embedding = model.encode(data['content']).tolist()
    embeddings.append(embedding)
    metadatas.append({'source': data['file_name']})
    ids.append(str(index + 1))

In [23]:
# create the new chromaDB and use embeddings to add and query data
pet_collection_emb = client.create_collection("pet_collection_emb")
pet_collection_emb.add(
    documents=documents,
    embeddings=embeddings,
    metadatas=metadatas,
    ids=ids
)

In [25]:
query = "What are the different kinds of pets people commonly own?"
input_em = model.encode(query).tolist()

results = pet_collection_emb.query(
    query_embeddings=[input_em],
    n_results=1
)
results

{'ids': [['4']],
 'embeddings': None,
 'documents': [['Pet animals come in all shapes and sizes, each suited to different lifestyles and home environments. Dogs and cats are the most common, known for their companionship and unique personalities. Small mammals like hamsters, guinea pigs, and rabbits are often chosen for their low maintenance needs. Birds offer beauty and song, and reptiles like turtles and lizards can make intriguing pets. Even fish, with their calming presence, can be wonderful pets.']],
 'metadatas': [[{'source': 'Different Types of Pet Animals.txt'}]],
 'distances': [[12.040446281433105]]}

In [26]:
query = "How to take care of the dogs?"
input_em = model.encode(query).tolist()

results = pet_collection_emb.query(
    query_embeddings=[input_em],
    n_results=1
)
results

{'ids': [['2']],
 'embeddings': None,
 'documents': [['Routine health care is crucial for pets to live long, happy lives. Regular vet check-ups help catch potential issues early and keep vaccinations up to date. Dental care is also essential to prevent diseases in pets, especially in dogs and cats. Regular grooming, parasite control, and weight management are other important aspects of pet health care.']],
 'metadatas': [[{'source': 'Health Care for Pets.txt'}]],
 'distances': [[14.663804054260254]]}

In [27]:
query = "foods that are recommended for dogs?"
input_em = model.encode(query).tolist()

results = pet_collection_emb.query(
    query_embeddings=[input_em],
    n_results=1
)
results

{'ids': [['1']],
 'embeddings': None,
 'documents': [['Proper nutrition is vital for the health and wellbeing of pets. Dogs and cats require a balanced diet that includes proteins, carbohydrates, and fats. Some may even have specific dietary needs based on their breed or age. Birds typically thrive on a diet of seeds, fruits, and vegetables, while reptiles have diverse diets ranging from live insects to fresh produce. Fish diets depend greatly on the species, with some needing live food and others subsisting on flakes or pellets.']],
 'metadatas': [[{'source': 'Nutrition Needs of Pet Animals.txt'}]],
 'distances': [[17.143932342529297]]}